#### Imports and Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.callbacks  import EarlyStopping
from keras.models import Sequential
from keras.layers import Lambda, SimpleRNN, Dense

import IPython, IPython.display, os, datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ts_utils

mpl.rcParams['figure.figsize'] = (14, 4)
mpl.rcParams['axes.grid'] = True

In [ ]:
# Read the data
df, df_scaled_trn, df_scaled_tst, scaler = ts_utils.load_file()
print("Training Data:")
display(df_scaled_trn)

input_slice  = slice(0,3)
label_slice  = slice(1,3)
window_len   = 3
ouput_len    = 3
batch_size   = 5

feat_op_len  = label_slice.stop - (label_slice.start or 0)
model_op_len = feat_op_len * ouput_len

ds_trn = tf.data.Dataset.from_tensor_slices(df_scaled_trn[df_scaled_trn.columns[input_slice]])
ds_tst = tf.data.Dataset.from_tensor_slices(df_scaled_tst[df_scaled_trn.columns[input_slice]])
window_trn = ts_utils.window(ds_trn, window_len, ouput_len, label_slice, batch_size=batch_size, skip=1)
window_tst = ts_utils.window(ds_tst, window_len, ouput_len, label_slice, batch_size=batch_size, skip=1)

#print("\n\nSample Window to verify the window is working:\n")
#for w in window_trn.take(3):
#    print(f"{w[0].numpy().shape}\n{w[0].numpy()}\n=>:{w[1].numpy().shape}\n{w[1].numpy()} \n")


In [ ]:
'''
    yh: [batch, time, features length]
'''
def inv_transform(yh, scaler, label_slice):
    yy=np.zeros([yh.shape[0], yh.shape[1], scaler.n_features_in_])
    yy[:, :, label_slice] = yh[0]
    ys = []
    for i in range(yh.shape[0]):
        yi = scaler.inverse_transform(yy[i])
        ys.append(yi[:, label_slice])
    return np.stack(ys)
    
#inv_transform(yh, scaler, label_slice)

def predict(model, window_trn, n=1):
    for w in window_trn.take(n):
        x = w[0]
        yh = model.predict(x)
        print(x.shape, x, "\n\n", yh, yh.shape)


In [ ]:
def compile_fit(model, window_trn, window_tst= None, opt=None, patience=3, epochs=1):
    early_stop = EarlyStopping(monitor='val_loss', patience=patience, mode='min')

    loss = tf.keras.losses.MeanSquaredError()
    opt  = opt or tf.keras.optimizers.Adam()
    mets = [tf.keras.metrics.MeanAbsoluteError()]

    ##=> Other options you can try
    #learning_rate = 1e-6
    #opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    #opt = tf.keras.optimizers.SGD()
    #loss=tf.keras.losses.Huber()

    model.compile(loss= loss, optimizer= opt, metrics=mets)

    history = []
    if (window_trn is not None):
        history = model.fit(window_trn, epochs=epochs, validation_data=window_tst, callbacks=[early_stop])

    return model, history

performance = {}

In [ ]:
srnn_model = tf.keras.models.Sequential([
    tf.keras.layers.SimpleRNN(40),
    tf.keras.layers.Dense(model_op_len),
    tf.keras.layers.Reshape([ouput_len, feat_op_len])
], name="SimpleRNN")

for opt in "sgd adam".split():
    model, history = compile_fit(srnn_model, window_trn, window_tst, epochs=1, opt=opt )
    performance[srnn_model.name + f":{opt}"] = ts_utils.eval_performance(srnn_model, window_trn, window_tst)


In [ ]:
# Finally plot the performance of the models
import ts_utils
performance = ts_utils.plot_performance([], window_trn, window_tst, performance=performance)
#print(performance)

In [ ]:
history.history, performance